In [26]:
import matplotlib.pyplot as plt
import numpy as np
import math

from qiskit import *
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT
from qiskit.circuit.library import MCXGate
from qiskit.circuit.gate import Gate

# QFT Adder (run before Grover/partial/Mizel)

In [27]:
#adds the number q_7q_6q_5q_4 to the number q_3q_2q_1q_0
qft_add = QuantumCircuit(8,8)
qft_add.append(QFT(4, do_swaps=True),[0,1,2,3])
qft_add.crz(np.pi/8,4,0)
qft_add.crz(np.pi/4,4,1)
qft_add.crz(np.pi/2,4,2)
qft_add.crz(np.pi,4,3)
qft_add.crz(np.pi/4,5,0)
qft_add.crz(np.pi/2,5,1)
qft_add.crz(np.pi,5,2)
qft_add.crz(np.pi/2,6,0)
qft_add.crz(np.pi,6,1)
qft_add.crz(np.pi,7,0)

qft_add.append(QFT(4, do_swaps=True).inverse(),[0,1,2,3])

In [28]:
qft_add.decompose().draw() # decomposition of the qft adder circuit has O(n^2) circuit depth,
                           # where n is the number of qubits used for representing each number

┌───┐»
q_0: ──────────────────────────────────────■─────────────■────────■───────┤ H ├»
                                           │       ┌───┐ │        │P(π/2) └───┘»
q_1: ────────────────────■────────■────────┼───────┤ H ├─┼────────■─────────X──»
                   ┌───┐ │        │P(π/2)  │       └───┘ │P(π/4)            │  »
q_2: ──────■───────┤ H ├─┼────────■────────┼─────────────■──────────────────X──»
     ┌───┐ │P(π/2) └───┘ │P(π/4)           │P(π/8)                             »
q_3: ┤ H ├─■─────────────■─────────────────■───────────────────────────────────»
     └───┘                                                                     »
q_4: ──────────────────────────────────────────────────────────────────────────»
                                                                               »
q_5: ──────────────────────────────────────────────────────────────────────────»
                                                                               »
q_6: ──────────────────────────────────────────────────────────────────────────»
                                                                               »
q_7: ──────────────────────────────────────────────────────────────────────────»
                                                                               »
c: 8/══════════════════════════════════════════════════════════════════════════»
                                                                               »
«        ┌──────────┐┌───┐┌───────────┐┌───┐┌─────────┐            ┌───┐»
«q_0: ─X─┤ U1(π/16) ├┤ X ├┤ U1(-π/16) ├┤ X ├┤ U1(π/8) ├────────────┤ X ├»
«      │ ├─────────┬┘└─┬─┘└───────────┘└─┬─┘└──┬───┬──┘┌──────────┐└─┬─┘»
«q_1: ─┼─┤ U1(π/8) ├───┼─────────────────┼─────┤ X ├───┤ U1(-π/8) ├──┼──»
«      │ ├─────────┤   │                 │     └─┬─┘   └──────────┘  │  »
«q_2: ─┼─┤ U1(π/4) ├───┼─────────────────┼───────┼───────────────────┼──»
«      │ ├─────────┤   │                 │       │                   │  »
«q_3: ─X─┤ U1(π/2) ├───┼─────────────────┼───────┼───────────────────┼──»
«        └─────────┘   │                 │       │                   │  »
«q_4: ─────────────────■─────────────────■───────■───────────────────┼──»
«                                                                    │  »
«q_5: ───────────────────────────────────────────────────────────────■──»
«                                                                       »
«q_6: ──────────────────────────────────────────────────────────────────»
«                                                                       »
«q_7: ──────────────────────────────────────────────────────────────────»
«                                                                       »
«c: 8/══════════════════════════════════════════════════════════════════»
«                                                                       »
«     ┌──────────┐           ┌───┐┌─────────┐                  ┌───┐»
«q_0: ┤ U1(-π/8) ├───────────┤ X ├┤ U1(π/4) ├──────────────────┤ X ├»
«     └──┬───┬───┘┌─────────┐└─┬─┘└─────────┘ ┌───┐┌──────────┐└─┬─┘»
«q_1: ───┤ X ├────┤ U1(π/4) ├──┼──────────────┤ X ├┤ U1(-π/4) ├──┼──»
«        └─┬─┘    └──┬───┬──┘  │  ┌──────────┐└─┬─┘└──┬───┬───┘  │  »
«q_2: ─────┼─────────┤ X ├─────┼──┤ U1(-π/4) ├──┼─────┤ X ├──────┼──»
«          │         └─┬─┘     │  └──────────┘  │     └─┬─┘      │  »
«q_3: ─────┼───────────┼───────┼────────────────┼───────┼────────┼──»
«          │           │       │                │       │        │  »
«q_4: ─────■───────────■───────┼────────────────┼───────■────────┼──»
«                              │                │                │  »
«q_5: ─────────────────────────■────────────────■────────────────┼──»
«                                                                │  »
«q_6: ───────────────────────────────────────────────────────────■──»
«                                                                   »
«q_7: ──────────────────────────────────────────────────────────────»
«        

# Grover

In [29]:
#this block implements the oracle that flips the phases of the target states
orcl_g = QuantumCircuit(17,17)
orcl_g.append(qft_add, range(8), range(8))       #a := a + b
orcl_g.append(qft_add, range(4,12), range(4,12)) #b := b + c
orcl_g.append(qft_add, range(8,16), range(8,16)) #c := c + d
orcl_g.x([0,2,3,4,6,7,8,10,11]) # this command regulates the target sums. currently: a + b = b + c = c + d = 2

u_phase = QuantumCircuit(17, 17)
c12x = MCXGate(12)
u_phase.h(16)                                         #|
u_phase.append(c12x, [0,1,2,3,4,5,6,7,8,9,10,11,16])  #|flip the phase of the target states
u_phase.h(16)                                         #|

orcl_g = orcl_g + u_phase + orcl_g.inverse()
#orcl_g.decompose().draw()

In [30]:
# implementation of the (partial) diffusion operator
diff = QuantumCircuit(17,17)

diff.h(range(3))                #|
diff.h(range(4,7))              #|map equal superposition to |0>^n state (auxiliary qubits are ignored)
diff.h(range(8,11))             #|
diff.h(range(12,15))            #|

diff.x(range(3))                #|
diff.x(range(4,7))              #|map |0>^n state to |1>^n state
diff.x(range(8,11))             #|
diff.x(range(12,15))            #|

diff.h(16)                                         #|
diff.append(c12x,[0,1,2,4,5,6,8,9,10,12,13,14,16]) #|flip the phase of the |1>^n+1 state
diff.h(16)                                         #|

diff.x(range(3))                #|
diff.x(range(4,7))              #|map |1>^n state to |0>^n state
diff.x(range(8,11))             #|
diff.x(range(12,15))            #|

diff.h(range(3))                #|
diff.h(range(4,7))              #|map |0>^n state to equal superposition
diff.h(range(8,11))             #|
diff.h(range(12,15))            #|

diff.draw()

┌───┐┌───┐     ┌───┐┌───┐
 q_0: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      ├───┤├───┤  │  ├───┤├───┤
 q_1: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      ├───┤├───┤  │  ├───┤├───┤
 q_2: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      └───┘└───┘  │  └───┘└───┘
 q_3: ────────────┼────────────
      ┌───┐┌───┐  │  ┌───┐┌───┐
 q_4: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      ├───┤├───┤  │  ├───┤├───┤
 q_5: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      ├───┤├───┤  │  ├───┤├───┤
 q_6: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      └───┘└───┘  │  └───┘└───┘
 q_7: ────────────┼────────────
      ┌───┐┌───┐  │  ┌───┐┌───┐
 q_8: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      ├───┤├───┤  │  ├───┤├───┤
 q_9: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      ├───┤├───┤  │  ├───┤├───┤
q_10: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      └───┘└───┘  │  └───┘└───┘
q_11: ────────────┼────────────
      ┌───┐┌───┐  │  ┌───┐┌───┐
q_12: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      ├───┤├───┤  │  ├───┤├───┤
q_13: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      ├───┤├───┤  │  ├───┤├───┤
q_14: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      └───┘└───┘  │  └───┘└───┘
q_15: ────────────┼────────────
      ┌───┐     ┌─┴─┐┌───┐     
q_16: ┤ H ├─────┤ X ├┤ H ├─────
      └───┘     └───┘└───┘     
c: 17/═════════════════════════

In [31]:
grov = QuantumCircuit(17,17)
grov.h([0,1,2,4,5,6,8,9,10,12,13,14]) #initialize variables
grov.x(16)                            #initialize ancilla
for n in range(29):                   #set number of iterations
    grov = grov + orcl_g + diff
#i.x(16)                              #|uncommenting these lines corresponds to defining ancilla to be in the |+>
#i.h(16)                              #|state in the target state (other needed changes would cancel each other out)
grov.measure(range(17), range(17))

In [32]:
provider = IBMQ.load_account() # alternatively you can simulate locally
backend = provider.get_backend('ibmq_qasm_simulator')

ibmqfactory.load_account:WARNING:2023-02-15 05:18:54,700: Credentials are already in use. The existing account in the session will be replaced.


In [33]:
transpiled_g = transpile(grov, backend=backend)

In [34]:
job_g = backend.run(transpiled_g)
retrieved_job_g = backend.retrieve_job(job_g.job_id())

In [35]:
print(retrieved_job_g.result().get_counts()) # least significant 4 bits := a,...,most significant bit := ancilla

{'10000001000000010': 332, '10001000100010001': 347, '10010000000100000': 345}


# Partial Diffusion

In [11]:
#this block implements the oracle that marks the the target states with ancilla = |1>
orcl_p = QuantumCircuit(17,17)
orcl_p.append(qft_add, range(8), range(8))       #a := a + b
orcl_p.append(qft_add, range(4,12), range(4,12)) #b := b + c
orcl_p.append(qft_add, range(8,16), range(8,16)) #c := c + d
orcl_p.x([0,2,3,4,6,7,8,10,11]) # this command regulates the target sums. currently: a + b = b + c = c + d = 2


u_mark = QuantumCircuit(17, 17)
c12x = MCXGate(12)
u_mark.append(c12x, [0,1,2,3,4,5,6,7,8,9,10,11,16])  # mark target states with ancilla = |1>

orcl_p = orcl_p + u_mark + orcl_p.inverse()
#orcl_p.decompose().draw()

┌──────┐┌─────────┐           ┌─────────┐                      »
 q_0: ┤0     ├┤ RZ(π/8) ├───────────┤ RZ(π/4) ├──────────────────────»
      │      │└────┬────┘┌─────────┐└────┬────┘           ┌─────────┐»
 q_1: ┤1     ├─────┼─────┤ RZ(π/4) ├─────┼────────────────┤ RZ(π/2) ├»
      │  qft │     │     └────┬────┘     │     ┌─────────┐└────┬────┘»
 q_2: ┤2     ├─────┼──────────┼──────────┼─────┤ RZ(π/2) ├─────┼─────»
      │      │     │          │          │     └────┬────┘     │     »
 q_3: ┤3     ├─────┼──────────┼──────────┼──────────┼──────────┼─────»
      └──────┘     │          │          │          │          │     »
 q_4: ─────────────■──────────■──────────┼──────────■──────────┼─────»
                                         │                     │     »
 q_5: ───────────────────────────────────■─────────────────────■─────»
                                                                     »
 q_6: ───────────────────────────────────────────────────────────────»
                                                                     »
 q_7: ───────────────────────────────────────────────────────────────»
                                                                     »
 q_8: ───────────────────────────────────────────────────────────────»
                                                                     »
 q_9: ───────────────────────────────────────────────────────────────»
                                                                     »
q_10: ───────────────────────────────────────────────────────────────»
                                                                     »
q_11: ───────────────────────────────────────────────────────────────»
                                                                     »
q_12: ───────────────────────────────────────────────────────────────»
                                                                     »
q_13: ───────────────────────────────────────────────────────────────»
                                                                     »
q_14: ───────────────────────────────────────────────────────────────»
                                                                     »
q_15: ───────────────────────────────────────────────────────────────»
                                                                     »
q_16: ───────────────────────────────────────────────────────────────»
                                                                     »
c: 17/═══════════════════════════════════════════════════════════════»
                                                                     »
«      ┌─────────┐                           ┌───────┐┌───────┐┌───────────┐»
« q_0: ┤ RZ(π/2) ├───────────────────────────┤ RZ(π) ├┤0      ├┤ U3(π,0,π) ├»
«      └────┬────┘                  ┌───────┐└───┬───┘│       │└───────────┘»
« q_1: ─────┼───────────────────────┤ RZ(π) ├────┼────┤1      ├─────────────»
«           │              ┌───────┐└───┬───┘    │    │  iqft │┌───────────┐»
« q_2: ─────┼──────────────┤ RZ(π) ├────┼────────┼────┤2      ├┤ U3(π,0,π) ├»
«           │     ┌───────┐└───┬───┘    │        │    │       │├───────────┤»
« q_3: ─────┼─────┤ RZ(π) ├────┼────────┼────────┼────┤3      ├┤ U3(π,0,π) ├»
«           │     └───┬───┘    │        │        │    └┬──────┤└┬─────────┬┘»
« q_4: ─────┼─────────■────────┼────────┼────────┼─────┤0     ├─┤ RZ(π/8) ├─»
«           │                  │        │        │     │      │ └────┬────┘ »
« q_5: ─────┼──────────────────■────────┼────────┼─────┤1     ├──────┼──────»
«           │                           │        │     │  qft │      │      »
« q_6: ─────■───────────────────────────■────────┼─────┤2     ├──────┼──────»
«                                                │     │      │      │      »
« q_7: ──────────────────────────────────────────■─────┤3     ├──────┼──────»
«                                                      └──────┘      │      »
« q_8: ──────────────────────────────────────────────

In [12]:
# implementation of the partial diffusion operator (diffuses in ancilla = |0> subspaces)
partial = QuantumCircuit(17,17)

partial.h(range(3))                #|
partial.h(range(4,7))              #|map equal superposition to |0>^n state (auxiliary qubits are ignored)
partial.h(range(8,11))             #|
partial.h(range(12,15))            #|

partial.x(range(3))                #|
partial.x(range(4,7))              #|map |0>^n state to |1>^n state
partial.x(range(8,11))             #|
partial.x(range(12,15))            #|

partial.x(16)                                         #|
partial.h(16)                                         #|
partial.append(c12x,[0,1,2,4,5,6,8,9,10,12,13,14,16]) #|flip the phase of the |1>^n|0> state
partial.h(16)                                         #|
partial.x(16)                                         #|

partial.x(range(3))                #|
partial.x(range(4,7))              #|map |1>^n state to |0>^n state
partial.x(range(8,11))             #|
partial.x(range(12,15))            #|

partial.h(range(3))                #|
partial.h(range(4,7))              #|map |0>^n state to equal superposition
partial.h(range(8,11))             #|
partial.h(range(12,15))            #|

partial.draw()

┌───┐┌───┐     ┌───┐┌───┐
 q_0: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      ├───┤├───┤  │  ├───┤├───┤
 q_1: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      ├───┤├───┤  │  ├───┤├───┤
 q_2: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      └───┘└───┘  │  └───┘└───┘
 q_3: ────────────┼────────────
      ┌───┐┌───┐  │  ┌───┐┌───┐
 q_4: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      ├───┤├───┤  │  ├───┤├───┤
 q_5: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      ├───┤├───┤  │  ├───┤├───┤
 q_6: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      └───┘└───┘  │  └───┘└───┘
 q_7: ────────────┼────────────
      ┌───┐┌───┐  │  ┌───┐┌───┐
 q_8: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      ├───┤├───┤  │  ├───┤├───┤
 q_9: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      ├───┤├───┤  │  ├───┤├───┤
q_10: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      └───┘└───┘  │  └───┘└───┘
q_11: ────────────┼────────────
      ┌───┐┌───┐  │  ┌───┐┌───┐
q_12: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      ├───┤├───┤  │  ├───┤├───┤
q_13: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      ├───┤├───┤  │  ├───┤├───┤
q_14: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
      └───┘└───┘  │  └───┘└───┘
q_15: ────────────┼────────────
      ┌───┐┌───┐┌─┴─┐┌───┐┌───┐
q_16: ┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├
      └───┘└───┘└───┘└───┘└───┘
c: 17/═════════════════════════

In [13]:
pd = QuantumCircuit(17,17)
pd.h([0,1,2,4,5,6,8,9,10,12,13,14]) # initialize variables

for n in range(41):                 # set number of iterations
    pd = pd + orcl_p + partial

#pd.h(16)                           # uncomment to map ancilla from |-> state to |1> state after reaching the target
pd.measure(range(17), range(17))

In [14]:
provider = IBMQ.load_account() # alternatively you can simulate locally
backend = provider.get_backend('ibmq_qasm_simulator')

ibmqfactory.load_account:WARNING:2023-02-15 05:10:04,656: Credentials are already in use. The existing account in the session will be replaced.


In [15]:
transpiled_pd = transpile(pd, backend=backend)

In [16]:
job_pd = backend.run(transpiled_pd)
retrieved_job_pd = backend.retrieve_job(job_pd.job_id())

In [17]:
print(retrieved_job_pd.result().get_counts()) # least significant 4 bits := a,...,most significant bit := ancilla

{'10000001000000010': 179, '00001000100010001': 164, '10001000100010001': 152, '10010000000100000': 172, '00000001000000010': 169, '00010000000100000': 188}


# Mizel

In [18]:
#this function returns an iteration of Mizel's algorithm for alpha = a
def mza(a):
    mz = QuantumCircuit(17, 17)
    
    mz.append(qft_add, range(8), range(8))        # a := a + b
    mz.append(qft_add, range(4,12), range(4,12))  # b := b + c
    mz.append(qft_add, range(8,16), range(8,16))  # c := c + d
    
    mz.ry(a/2,16)                                      #|
                                                       #|
    mz.x([0,2,3,4,6,7,8,10,11])                        #|
    c12x = MCXGate(12)                                 #| target-state-controlled ry(a) on the ancilla +
    mz.h(16)                                           #| flip the state of the target state
    mz.append(c12x, [0,1,2,3,4,5,6,7,8,9,10,11,16])    #|
    mz.h(16)                                           #|
                                                       #|
    mz.ry(-a/2,16)                                     #|
    
    mz.x([0,2,3,4,6,7,8,10,11])
    mz.append(qft_add.inverse(), range(8,16), range(8,16))
    mz.append(qft_add.inverse(), range(4,12), range(4,12))
    mz.append(qft_add.inverse(), range(8), range(8))

    mz.h(range(3))                                     #|
    mz.h(range(4,7))                                   #|
    mz.h(range(8,11))                                  #|
    mz.h(range(12,15))                                 #|
                                                       #|
    mz.x(range(3))                                     #|
    mz.x(range(4,7))                                   #|
    mz.x(range(8,11))                                  #|
    mz.x(range(12,15))                                 #|
                                                       #|
    mz.h(16)                                           #| partial diffusion on the ancilla = |1> subspace
    mz.append(c12x,[0,1,2,4,5,6,8,9,10,12,13,14,16])   #| (corresponds to ancilla controlled diffusion)
    mz.h(16)                                           #|
                                                       #|
    mz.x(range(3))                                     #|
    mz.x(range(4,7))                                   #|
    mz.x(range(8,11))                                  #|
    mz.x(range(12,15))                                 #|
                                                       #|
    mz.h(range(3))                                     #|
    mz.h(range(4,7))                                   #|
    mz.h(range(8,11))                                  #|
    mz.h(range(12,15))                                 #|

    return mz

In [19]:
qr = QuantumRegister(17)
cr = [ClassicalRegister(1) for _ in range(17)]
mzl = QuantumCircuit(qr)
for r in cr:
    mzl.add_register(r)

In [20]:
mzl.h([0,1,2,4,5,6,8,9,10,12,13,14]) # initialize variables
mzl.x(16)                            # initialize ancilla
mzl.append(mza(np.pi/2),range(17), range(17))   # append the first iteration with alpha = pi/2
for n in range(2,75):
    mzl.measure(16,16)               # measure the ancilla
    mzl.append(mza((1-math.sin(np.pi/(2*n)))/(1+math.sin(np.pi/(2*n)))),range(17),range(17)).c_if(cr[16], 1) # <--.
                                  #                                                                               |
                                  # append the next iteration conditioned on the measurement outcome of the ancilla
mzl.measure(range(17), range(17))

In [22]:
provider = IBMQ.load_account() # alternatively you can simulate locally
backend = provider.get_backend('ibmq_qasm_simulator')

ibmqfactory.load_account:WARNING:2023-02-15 05:10:47,050: Credentials are already in use. The existing account in the session will be replaced.


In [23]:
transpiled_mzl = transpile(mzl, backend=backend)

In [24]:
job_mzl = backend.run(transpiled_mzl)
retrieved_job_mzl = backend.retrieve_job(job_mzl.job_id())

In [25]:
print(retrieved_job_mzl.result().get_counts()) # least significant 4 bits := a,...,most significant bit := ancilla

{'0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1': 332, '1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1': 1, '1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0': 1, '1 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0': 1, '1 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 1': 1, '1 0 1 1 0 0 1 1 0 0 0 0 1 0 0 1 1': 1, '1 0 1 1 1 0 0 1 1 0 1 1 0 0 1 1 1': 1, '0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0': 342, '0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0': 344}


# subtraction with classical approach (not used above)

In [155]:
# here is an example of a two number subtracter implemented with a classical approach
# it subtracts the number q_5q_4q_3 fromthe number q_2q_1q_0
# we do not use this approacch in the above implemented algorithms. this is only here for illustration purposes.
sub = QuantumCircuit(6,6)
c3x = MCXGate(3)
c2x = MCXGate(2)

sub.x(range(2))
sub.append(c3x,[0,1,3,2])
sub.x(1)
sub.append(c2x,[0,3,1])
sub.x(0)
sub.cx(3,0)
sub.x(1)
sub.append(c2x,[1,4,2])
sub.x(1)
sub.cx(4,1)
sub.cx(5,2)

sub.draw()

┌───┐               ┌───┐┌───┐               
q_0: ┤ X ├──■─────────■──┤ X ├┤ X ├───────────────
     ├───┤  │  ┌───┐┌─┴─┐├───┤└─┬─┘     ┌───┐┌───┐
q_1: ┤ X ├──■──┤ X ├┤ X ├┤ X ├──┼────■──┤ X ├┤ X ├
     └───┘┌─┴─┐└───┘└─┬─┘└───┘  │  ┌─┴─┐├───┤└─┬─┘
q_2: ─────┤ X ├───────┼─────────┼──┤ X ├┤ X ├──┼──
          └─┬─┘       │         │  └─┬─┘└─┬─┘  │  
q_3: ───────■─────────■─────────■────┼────┼────┼──
                                     │    │    │  
q_4: ────────────────────────────────■────┼────■──
                                          │       
q_5: ─────────────────────────────────────■───────
                                                  
c: 6/═════════════════════════════════════════════

In [159]:
sub.decompose().decompose().draw() # decomposition of the subtraction circuit has O(n^3) circuit depth,
                                   # where n is the number of qubits used for representing each number

┌──────────┐┌────────┐                                                    »
q_0: ┤ U(π,0,π) ├┤ P(π/8) ├──■───────────────■────────────────────■────────────»
     ├──────────┤├────────┤┌─┴─┐┌─────────┐┌─┴─┐                  │            »
q_1: ┤ U(π,0,π) ├┤ P(π/8) ├┤ X ├┤ P(-π/8) ├┤ X ├──■───────────────┼────────────»
     └──┬───┬───┘├────────┤└───┘└─────────┘└───┘  │               │            »
q_2: ───┤ H ├────┤ P(π/8) ├───────────────────────┼───────────────┼────────────»
      ┌─┴───┴──┐ └────────┘                     ┌─┴─┐┌─────────┐┌─┴─┐┌────────┐»
q_3: ─┤ P(π/8) ├────────────────────────────────┤ X ├┤ P(-π/8) ├┤ X ├┤ P(π/8) ├»
      └────────┘                                └───┘└─────────┘└───┘└────────┘»
q_4: ──────────────────────────────────────────────────────────────────────────»
                                                                               »
q_5: ──────────────────────────────────────────────────────────────────────────»
                                                                               »
c: 6/══════════════════════════════════════════════════════════════════════════»
                                                                               »
«                                                                              »
«q_0: ──────────────────■───────────────────────────────────────────────────■──»
«                       │                                                   │  »
«q_1: ──■───────────────┼────────────────────■──────────────────────────────┼──»
«       │               │  ┌───┐┌─────────┐┌─┴─┐┌────────┐┌───┐┌─────────┐┌─┴─┐»
«q_2: ──┼───────────────┼──┤ X ├┤ P(-π/8) ├┤ X ├┤ P(π/8) ├┤ X ├┤ P(-π/8) ├┤ X ├»
«     ┌─┴─┐┌─────────┐┌─┴─┐└─┬─┘└─────────┘└───┘└────────┘└─┬─┘└─────────┘└───┘»
«q_3: ┤ X ├┤ P(-π/8) ├┤ X ├──■──────────────────────────────■──────────────────»
«     └───┘└─────────┘└───┘                                                    »
«q_4: ─────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q_5: ─────────────────────────────────────────────────────────────────────────»
«                                                                              »
«c: 6/═════════════════════════════════════════════════════════════════════════»
«                                                                              »
«                                                                           »
«q_0: ──────────────────────────────────────────────────────────────────────»
«                                    ┌──────────┐┌─────────┐           ┌───┐»
«q_1: ────────────────────────────■──┤ U(π,0,π) ├┤ U2(0,π) ├───────────┤ X ├»
«     ┌────────┐┌───┐┌─────────┐┌─┴─┐└┬────────┬┘└──┬───┬──┘┌─────────┐└─┬─┘»
«q_2: ┤ P(π/8) ├┤ X ├┤ P(-π/8) ├┤ X ├─┤ P(π/8) ├────┤ X ├───┤ P(-π/8) ├──┼──»
«     └────────┘└─┬─┘└─────────┘└───┘ └────────┘    └─┬─┘   └─────────┘  │  »
«q_3: ────────────■───────────────────────────────────■──────────────────■──»
«                                                                           »
«q_4: ──────────────────────────────────────────────────────────────────────»
«                                                                           »
«q_5: ──────────────────────────────────────────────────────────────────────»
«                                                                           »
«c: 6/══════════════════════════════════════════════════════════════════════»
«                                                                           »
«                                                                             »
«q_0: ──■───────────────────■───────────────────────────────────■─────────────»
«       │  ┌──────────┐   ┌─┴─┐   ┌─────────┐┌───┐┌──────────┐┌─┴─┐┌─────────┐»
«q_1: ──┼──┤ U1(-π/4) ├───┤ X ├───┤ U1(π/4) ├┤ X ├┤ U1(-π/4) ├┤ X ├┤ U1(π/4) ├»
«     ┌─┴─┐└──┬───┬───┘┌──┴───┴──┐└──┬───┬──┘└─┬─┘├──────────┤└───┘└─────────┘»
«q_2: